In [1]:
import re
import random
import time
from tqdm.auto import tqdm
from html import unescape
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

In [2]:
from pprint import pprint

In [3]:
session = requests.session()
ua = UserAgent(verify_ssl=False)  # ради безопасности

#### Лента

In [5]:
def parsedate(date):
    url = f'https://lenta.ru{date}'
    response = session.get(url, headers={'User-Agent': ua.random})
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    main = soup.find('section', {'class':'b-layout js-layout b-layout_archive'})
    next_l = main.find_all('a', {'class':'control_mini'})[1].get('href')
    
    news = main.find_all('div', {'class':'titles'})
    links = [newss.find('a').get('href') for newss in news]
    links = [link for link in links if link.startswith('/news')]
            
    return links, next_l

In [6]:
def parsenews(links, date):
    wholeday = ''

    for link in tqdm(links):
        url = f'https://lenta.ru{link}'
        response = session.get(url, headers={'User-Agent': ua.random})
        page = response.text
        soup = BeautifulSoup(page, 'html.parser')
        headline = soup.find('h1', {'itemprop':'headline'}).get_text()
        text = soup.find('div', {'itemprop':'articleBody'})
        passages = text.find_all('p')
        passages = [p.get_text() for p in passages]
        topic = soup.find('div', {'class':'b-topic__info'})
        gmtime = topic.find('time').get_text().split(',')[0]
        link_n = link.rsplit('/', 2)[1]
        time_n = gmtime.strip(' ')
        text_n = ' '.join(passages)
        wholenews = f'<news>\n<id>{link_n}</id>\n<time>{time_n}</time>\n<headline>{headline}</headline>\n<text>{text_n}</text>\n</news>\n'
        wholeday = wholeday + wholenews
        time.sleep(random.uniform(1.5, 2.2))
    
    filename = 'l-' + date.strip('/').replace('/', '-') + '.txt'
    
    with open(filename, 'a+', encoding='utf-8') as f:
        f.write(wholeday)

In [7]:
def get_news(date):
    links, next_date = parsedate(date)
    createtxt = parsenews(links, date)
    return next_date

In [26]:
next_date = '/2002/01/01/'
for i in range(4):
    next_date = get_news(next_date)

#### Взгляд

In [5]:
import sqlite3

In [170]:
conn = sqlite3.connect('vzglyad.db')  # структура БД
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS news
(year int, month int, day int, id_news int, time text, headline text, text text)
""")
conn.commit()
conn.close()

In [11]:
def updbd(idnew, amount):

    conn = sqlite3.connect('vzglyad.db')
    cur = conn.cursor()
    session = requests.session()
    ua = UserAgent(verify_ssl=False)
    
    for link in tqdm(range(amount)):
        url = f'https://vz.ru/news/2017/8/10/{idnew}.html'
        response = session.get(url, headers={'User-Agent': ua.random})
        page = response.text
        soup = BeautifulSoup(page, 'html.parser')
        
        meta = soup.find('meta', {'property':'og:url'})
        if meta:
            meta = meta.get('content', []).replace('<br>', '<br> ')
            if 'news' in meta:
    
                date = re.search(r'(\d{4})\/(\d\d?)\/(\d\d?)\/(\d+?)\.', meta)
                year = date.group(1)
                month = date.group(2)
                day = date.group(3)
                id_news = date.group(4)
    
                if soup.find('p', {'style':'float: left; margin: 0;  padding: 0'}):
                    gtime = soup.find('p',{'style':'float: left; margin: 0;  padding: 0'})
                    gtime = gtime.get_text()
                    gti = gtime.split(',')[1].strip(' ')
                elif soup.find('td', {'style':'padding:10px 0 10px 0;'}):
                    alt_time = soup.find('td', {'style':'padding:10px 0 10px 0;'}).get_text()
                    gti = re.search('\d{2}:\d{2}', alt_time).group()
                else:
                    gti = '-'
                    print(idnew)
    
                headline = soup.find('meta', {'property':'og:title'}).get('content')
                text = soup.find('div', {'class':'text newtext'}).get_text().replace('\n', ' ').strip(' ')
    
                cur.execute(
                    'INSERT INTO news VALUES (?, ?, ?, ?, ?, ?, ?)',
                    (year, month, day, id_news, gti, headline, text))
                conn.commit()
            
        idnew +=1
       
        time.sleep(random.uniform(1.3, 1.7))
        
    conn.close()

In [10]:
902183 - 900696

1487

In [12]:
updbd(900697, 1486) #902183